In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import keras

# Import resnet50 model from keras
from keras.applications.resnet import ResNet50, preprocess_input
from keras.layers import Input

# Image libraries
import PIL.Image

# Linear Algebra Libraries
import numpy as np

2023-03-06 22:18:07.963837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 22:18:08.105021: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-06 22:18:08.107867: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 22:18:08.107880: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
# Create resnet50 model for feature extraction
new_input = Input(shape=(28, 28, 3))
res = ResNet50(include_top = False, input_tensor = new_input)

In [ ]:
from keras.layers import Concatenate

In [ ]:
def create_batches(X, batch_size = 32):
    n_batches = X.shape[0] // batch_size
    n_batches += 1
    all_batches = []
    for batch in range(n_batches):
        start = batch * batch_size
        end   = start + batch_size
        images = X[start:end, :, :]
        preprocessed_images = []
        for image in images:
            image = np.array(PIL.Image.fromarray(image).convert("RGB"))
            image = image.reshape((1, *image.shape))
            preprocessed_images.append(image)
        batch_tensor = Concatenate(axis = 0)(preprocessed_images)
        all_batches.append(batch_tensor)
        break
    return all_batches

In [ ]:
btchs = create_batches(x_train)

In [ ]:
embeddings = res.predict(btchs[0])

1/1 [==============================] - 0s 74ms/step


In [ ]:
embeddings.shape

(32, 1, 1, 2048)